# Data Preprocessing
This notebook applies the following changes to the original Combined_Results and Senior_Commercial_Analyst files. The output will be a dataframe as a csv file for each of the three sheets in Combined Results and two sheets in Senior Analyst, as well as a csv called "CombinedResultsAll" and "SeniorAnalystAll" which merges the three and two respective sheets.

Summary of Changes Applied:
- Converted repeated rows into feature columns ('Label' and 'Skill' columns)
- Removed the '\10' from 'Self Rating' so it is numerical
- Changed percentages to a number out of 10 to match 'Self Rating'
- Removed duplicate rows with exact matching Candidate Test Ids/IDs (however duplicate IDs with differing features are left in)
- Converted time columns to Datetime type (disregarded time zone)
- Note: All Senior_Commercial_Analyst.xlsx candidates attempted test_id 1165 in Combined_Results.xlsx

In [46]:
import pandas as pd
import numpy as np

df1a = pd.read_excel('Combined Results Final.xlsx', sheet_name='Candidates')
df1b = pd.read_excel('Combined Results Final.xlsx', sheet_name='Skills')
df1c = pd.read_excel('Combined Results Final.xlsx', sheet_name='Pre Test Survey Info')
df2a = pd.read_excel('Senior Commercial Analyst (Amsterdam).xlsx', sheet_name='Candidates')
df2b = pd.read_excel('Senior Commercial Analyst (Amsterdam).xlsx', sheet_name='Skills')
df3 = pd.read_csv('Queries.csv')

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

df1b['Self Rating'] = df1b['Self Rating'].dt.strftime('%d/%m')
df1c.rename(columns={'candidate_test_id':'Candidate Test ID'}, inplace=True)

In [47]:
######################################
################ DF1A ################
######################################

# Convert columns to datetime type, ignore any errors that appear due to timezone.
df1a['Date Invited'] = pd.to_datetime(df1a['Date Invited']).dt.tz_localize(None)
df1a['Date Started'] = pd.to_datetime(df1a['Date Started']).dt.tz_localize(None)
df1a['Date Completed'] = pd.to_datetime(df1a['Date Completed']).dt.tz_localize(None)

# Convert Overall Score in percentage to score out of 10
df1a['Overall Score'] = df1a['Overall Score']*10

# Save dataframe as csv
df1a.to_csv('Combined_Results_Candidates_Final.csv')

df1a

C:\Users\fores\anaconda3\lib\site-packages\dateutil\parser\_parser.py:1213: UnknownTimezoneWarning: tzname AEST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
C:\Users\fores\anaconda3\lib\site-packages\dateutil\parser\_parser.py:1213: UnknownTimezoneWarning: tzname AEDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


,Candidate Test ID,Location,Date Invited,Date Started,Date Completed,Parts Completed,Overall Score
0,26924,Chile,2021-08-29 10:40:02,2021-08-30 11:35:45,2021-08-30 12:35:58,3,4.7
1,26925,NaN,2021-08-29 10:40:02,NaT,NaT,0,0.0
2,26926,NaN,2021-08-29 10:40:03,NaT,NaT,0,0.0
3,26927,Algeria,2021-08-29 10:40:04,2021-08-29 11:00:48,2021-08-29 11:33:47,3,2.5
4,26928,France,2021-08-29 10:40:04,2021-09-03 18:29:23,2021-09-03 19:32:14,3,4.3
...,...,...,...,...,...,...,...
5273,26708,NaN,2021-08-26 00:00:00,NaT,NaT,0,0.0
5274,26709,NaN,2021-08-26 00:00:00,NaT,NaT,0,0.0
5275,26710,NaN,2021-08-26 00:00:00,NaT,NaT,0,0.0
5276,26711,NaN,2021-08-26 00:00:00,NaT,NaT,0,0.0


In [48]:
######################################
################ DF1B ################
######################################

##Function that returns a dataframe filtered by the skill (i.e. Data Literacy, Statistics, SQL, ... )
def skill_extract(df, skill):
    df_slice = df[df['Skill'] == skill]
    df_slice = df_slice.rename(columns={'Score':skill+' score', 'Self Rating':skill+' Self Rating', 'Score Percentile':skill+' Score Percentile'}) #Rename columns for interpretability
    df_slice = df_slice.drop(columns='Skill')
    df_slice = df_slice.replace(to_replace='(\d+)\/10', value='\\1', regex=True) #Changes x/10 to numeric (just x)
    
    df_slice = df_slice.replace(to_replace='(\d+)[a-z]+', value='\\1', regex=True) #Changes 1st, 2nd, 3rd... to numeric (just x)

    df_slice = df_slice.astype({skill+' score': 'float', skill+' Self Rating':'float', skill+' Score Percentile':'float'}) #Convert columns to float
    df_slice[skill+' score'] = df_slice[skill+' score']*10 #Convert percentage to score out of 10
    return df_slice

##Create dfs, a dictionary of dataframes with keys being the skill and values being the dataframe filtered by skill
df1b_dict = {}
for skill in df1b['Skill'].unique():
    df1b_dict[skill] = skill_extract(df1b, skill).drop_duplicates(subset='Candidate Test ID', keep='first') # Keep the first since all duplicates after the first coincidentally only have missing/Nan values

print('A complete list of skills:', df1b_dict.keys())
    
#Uncomment to show an example of a df_slice
#df1b_dict['Chart Interpretation'] # This will have no duplicate Candidate Test Ids

## Joining all the dataframes in the dictionary 'df1b_dict' on Candidate Test Id
df1b = df1b[['Candidate Test ID']].drop_duplicates()
for key in df1b_dict:
    df1b = df1b.merge(df1b_dict[key], how='outer', on='Candidate Test ID', sort=True)

## Drop duplicate rows
df1b = df1b.drop_duplicates()

print('df1b has', len(df1b), 'rows and', len(df1b['Candidate Test ID'].unique()), 'unique Candidate Test Ids')

df1b.to_csv('Combined_Results_Skills_Final.csv') #Save dataframe as csv

df1b

A complete list of skills: dict_keys(['Business Acumen', 'SQL', 'Chart Interpretation', 'Reports & Visualisations', 'Data Literacy', 'Statistics', 'Relational Databases', 'Product Analytics', 'Data Management', 'Data Analysis', 'Machine Learning', 'Data Science', 'Source Control with Git', 'Python', 'R', 'Analytics Programming', 'Apache Spark', 'Data Modelling', 'Google Analytics', 'Digital Marketing Analytics'])
df1b has 5275 rows and 5275 unique Candidate Test Ids


,Candidate Test ID,Business Acumen score,Business Acumen Score Percentile,Business Acumen Self Rating,SQL score,SQL Score Percentile,SQL Self Rating,Chart Interpretation score,Chart Interpretation Score Percentile,Chart Interpretation Self Rating,Reports & Visualisations score,Reports & Visualisations Score Percentile,Reports & Visualisations Self Rating,Data Literacy score,Data Literacy Score Percentile,Data Literacy Self Rating,Statistics score,Statistics Score Percentile,Statistics Self Rating,Relational Databases score,Relational Databases Score Percentile,Relational Databases Self Rating,Product Analytics score,Product Analytics Score Percentile,Product Analytics Self Rating,Data Management score,Data Management Score Percentile,Data Management Self Rating,Data Analysis score,Data Analysis Score Percentile,Data Analysis Self Rating,Machine Learning score,Machine Learning Score Percentile,Machine Learning Self Rating,Data Science score,Data Science Score Percentile,Data Science Self Rating,Source Control with Git score,Source Control with Git Score Percentile,Source Control with Git Self Rating,Python score,Python Score Percentile,Python Self Rating,R score,R Score Percentile,R Self Rating,Analytics Programming score,Analytics Programming Score Percentile,Analytics Programming Self Rating,Apache Spark score,Apache Spark Score Percentile,Apache Spark Self Rating,Data Modelling score,Data Modelling Score Percentile,Data Modelling Self Rating,Google Analytics score,Google Analytics Score Percentile,Google Analytics Self Rating,Digital Marketing Analytics score,Digital Marketing Analytics Score Percentile,Digital Marketing Analytics Self Rating
0,12355,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.0,7.0,0.000,0.0,9.0,0.0,0.0,10.0,NaN,NaN,NaN,0.000,0.0,10.0,0.000,0.0,9.0,0.0,0.0,8.0,NaN,NaN,NaN,0.000,0.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,12397,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.667,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,2.857,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.333,NaN,NaN,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,12398,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,12399,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,12400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5270,31943,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5271,31944,NaN,NaN,NaN,8.750,93.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.444,67.0,8.0,NaN,NaN,NaN,6.0,80.0,7.0,NaN,NaN,NaN,6.552,85.0,9.0,3.103,27.0,9.0,7.5,81.0,9.0,5.0,45.0,7.0,7.143,73.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5272,31945,NaN,NaN,NaN,3.125,44.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.222,42.0,7.0,NaN,NaN,NaN,4.0,66.0,6.0,NaN,NaN,NaN,0.000,0.0,7.0,2.759,24.0,8.0,6.0,55.0,8.0,0.0,0.0,8.0,4.286,42.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5273,31946,NaN,NaN,NaN,NaN,NaN,NaN

In [50]:
######################################
################ DF1C ################
######################################

##Function that returns a dataframe filtered by the skill (i.e. Location, Years of experience, ... )
def skill_extract(df, skill):
    df_slice = df1c[df1c['label'] == skill]
    df_slice = df_slice.rename(columns={'value':skill})
    df_slice = df_slice.drop(columns=['label', 'test_id'])
    return df_slice

##Create dfs, a dictionary of dataframes with keys being the skill and values being the dataframe filtered by skill
df1c_dict = {}
for skill in df1c['label'].unique():
    df1c_dict[skill] = skill_extract(df1c, skill).drop_duplicates(subset='Candidate Test ID', keep='first') # Keep the first since all duplicates after the first coincidentally only have missing/Nan values

print('A complete list of labels:', df1c_dict.keys())
    
#Uncomment the line below to show an example of a df_slice
#df1c_dict['Year of Birth'] # This will have no duplicate Candidate Test Ids

## Joining all the dataframes in the dictionary 'df1c_dict' on Candidate Test Id
df1c_new = df1c[['Candidate Test ID', 'test_id']].drop_duplicates()
for key in df1c_dict:
    df1c_new = df1c_new.merge(df1c_dict[key], how='outer', on='Candidate Test ID', sort=True)

## Drop duplicate rows
df1c_new = df1c_new.drop_duplicates()

## Change column type from object to int
df1c_new = df1c_new.astype({'Years of Experience':'int'})

## Replace "I'd rather not say" with NaN values
df1c_new = df1c_new.replace(to_replace="I'd rather not say", value=np.nan, regex=True)

print('df1c_new has', len(df1c_new), 'rows and', len(df1c_new['Candidate Test ID'].unique()), 'unique Candidate Test IDs')

df1c_new.to_csv('Combined_Results_Pre_Test_Survey_Info_Final.csv') #Save dataframe as csv

df1c_new

A complete list of labels: dict_keys(['Location', 'Years of Experience', 'Current Industry', 'Gender', 'Year of Birth', 'Availability'])
df1c_new has 2588 rows and 2588 unique Candidate Test IDs


,Candidate Test ID,test_id,Location,Years of Experience,Current Industry,Gender,Year of Birth,Availability
0,12355,609,Canada,0,Accounting,Other,1950.0,NaN
1,12397,619,Algeria,9,Alternative Dispute Resolution,Other,2009.0,NaN
2,13625,619,Canada,13,"Health, Wellness and Fitness",Male,1983.0,NaN
3,13627,619,Canada,0,Oil & Energy,Male,1977.0,NaN
4,14122,619,Canada,3,Information Technology and Services,Male,1976.0,NaN
...,...,...,...,...,...,...,...,...
2583,31744,951,Australia,3,NaN,Male,1997.0,2-3 weeks notice
2584,31941,951,Australia,1,NaN,Male,1995.0,1-2 weeks notice
2585,31944,951,Australia,15,NaN,Male,1970.0,2-3 weeks notice
2586,31945,951,Australia,12,NaN,Male,1981.0,1 months notice


In [52]:
######################################
###### Combine DF1A, DF1B, DF1C ######
######################################

## Merge/join the three sheets/dataframes on Candidate Test Id
## Aside: We have only 933 out of 3091 Candidate Test Ids that appear in all three sheets. 
Combined_Results = df1a.merge(df1b.merge(df1c_new, how='outer', on='Candidate Test ID'), how='outer', on='Candidate Test ID')
Combined_Results = Combined_Results.drop(columns='Location_x')
Combined_Results = Combined_Results.rename(columns={'Location_y':'Location'})

## Export the merged dataframe as csv
Combined_Results.to_csv('Combined_Results_All_Final.csv') #Save dataframe as csv

## View all duplicate Candidate Test Ids
Combined_Results[Combined_Results['Candidate Test ID'].duplicated(keep=False)]

,Candidate Test ID,Date Invited,Date Started,Date Completed,Parts Completed,Overall Score,Business Acumen score,Business Acumen Score Percentile,Business Acumen Self Rating,SQL score,SQL Score Percentile,SQL Self Rating,Chart Interpretation score,Chart Interpretation Score Percentile,Chart Interpretation Self Rating,Reports & Visualisations score,Reports & Visualisations Score Percentile,Reports & Visualisations Self Rating,Data Literacy score,Data Literacy Score Percentile,Data Literacy Self Rating,Statistics score,Statistics Score Percentile,Statistics Self Rating,Relational Databases score,Relational Databases Score Percentile,Relational Databases Self Rating,Product Analytics score,Product Analytics Score Percentile,Product Analytics Self Rating,Data Management score,Data Management Score Percentile,Data Management Self Rating,Data Analysis score,Data Analysis Score Percentile,Data Analysis Self Rating,Machine Learning score,Machine Learning Score Percentile,Machine Learning Self Rating,Data Science score,Data Science Score Percentile,Data Science Self Rating,Source Control with Git score,Source Control with Git Score Percentile,Source Control with Git Self Rating,Python score,Python Score Percentile,Python Self Rating,R score,R Score Percentile,R Self Rating,Analytics Programming score,Analytics Programming Score Percentile,Analytics Programming Self Rating,Apache Spark score,Apache Spark Score Percentile,Apache Spark Self Rating,Data Modelling score,Data Modelling Score Percentile,Data Modelling Self Rating,Google Analytics score,Google Analytics Score Percentile,Google Analytics Self Rating,Digital Marketing Analytics score,Digital Marketing Analytics Score Percentile,Digital Marketing Analytics Self Rating,test_id,Location,Years of Experience,Current Industry,Gender,Year of Birth,Availability
994,28721,2021-09-13 13:41:28,2021-09-18 14:45:03,2021-09-18 15:50:43,3.0,3.8,0.0,0.0,7.0,0.000,0.0,8.0,4.762,58.0,9.0,5.385,0.0,9.0,7.273,81.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1165.0,Pakistan,2.0,Information Technology and Services,Male,1996.0,NaN
995,28721,2021-09-13 13:41:28,2021-09-18 14:45:03,2021-09-18 15:50:43,3.0,3.8,0.0,0.0,7.0,0.000,0.0,8.0,4.762,58.0,9.0,5.385,0.0,9.0,7.273,81.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1165.0,Pakistan,2.0,Information Technology and Services,Male,1996.0,NaN
1858,18881,2021-06-07 14:22:37,2021-06-08 09:05:03,2021-06-08 10:04:48,2.0,3.6,NaN,NaN,NaN,5.417,68.0,9.0,4.000,46.0,9.0,8.000,83.0,10.0,NaN,NaN,NaN,0.000,0.0,8.0,3.846,40.0,8.0,3.333,0.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,983.0,Australia,3.0,Information Technology and Services,Male,1995.0,NaN
1859,18881,2021-06-07 14:22:37,2021-06-08 09:05:03,2021-06-08 10:04:48,2.0,3.6,NaN,NaN,NaN,5.417,68.0,9.0,4.000,46.0,9.0,8.000,83.0,10.0,NaN,NaN,NaN,0.000,0.0,8.0,3.846,40.0,8.0,3.333,0.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,983.0,Australia,3.0,Information Technology and Services,Male,1995.0,NaN
2060,19352,2021-06-11 18:32:40,2021-06-14 09:59:44,2021-06-14 11:00:19,2.0,3.5,NaN,NaN,NaN,5.417,68.0,8.0,4.000,46.0,7.0,2.000,40.0,7.0,NaN,NaN,NaN,2.632,44.0,6.0,7.692,83.0,7.0,1.515,0.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,983.0,Australia,1.0,Information Technology and Services,Male,1996.0,NaN
2061,19352,2021-06-11 18:32:40,2021-06-14 09:59:44,2021-06-14 11:00:19,2.0,3.5,NaN,NaN,NaN,5.417,68.0,8.0,4.000,46.0,7.0,2.000,40.0,7.0,NaN,NaN,

In [3]:
df2a = pd.read_excel('Senior Commercial Analyst (Amsterdam) New.xlsx', sheet_name='Candidates')
df2b = pd.read_excel('Senior Commercial Analyst (Amsterdam) New.xlsx', sheet_name='Skills')

In [4]:
######################################
################ DF2A ################
######################################

df2a['Date Invited'] = pd.to_datetime(df2a['Date Invited']).dt.tz_localize(None)
df2a['Date Started'] = pd.to_datetime(df2a['Date Started']).dt.tz_localize(None)
df2a['Date Completed'] = pd.to_datetime(df2a['Date Completed']).dt.tz_localize(None)
df2a['Overall Score'] = df2a['Overall Score']*10

df2a.to_csv('Senior_Commercial_Analyst_(Amsterdam)_Candidates_New.csv') #Save dataframe as csv

C:\Users\fores\anaconda3\lib\site-packages\dateutil\parser\_parser.py:1213: UnknownTimezoneWarning: tzname AEST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


In [5]:
######################################
################ DF2B ################
######################################

##Function that returns a dataframe filtered by the skill (i.e. Data Literacy, Statistics, SQL, ... )
##Same as the one in df1b
def skill_extract(df, skill):
    df_slice = df[df['Skill'] == skill]
    df_slice = df_slice.rename(columns={'Score':skill+' score', 'Self Rating':skill+' Self Rating'}) #Rename columns for interpretability
    df_slice = df_slice.drop(columns='Skill')
    df_slice = df_slice.replace(to_replace='(\d+)\/10', value='\\1', regex=True) #Changes x/10 to numeric (just x)
    df_slice = df_slice.replace(to_replace='([0-9\.]+)\%', value='\\1', regex=True) #Removes percentage sign
    df_slice = df_slice.astype({skill+' score': 'float', skill+' Self Rating':'float'}) #Convert columns to float
    df_slice[skill+' score'] = df_slice[skill+' score']/10 #Convert percentage to score out of 10
    return df_slice

##Create dfs, a dictionary of dataframes with keys being the skill and values being the dataframe filtered by skill
df2b_dict = {}
for skill in df2b['Skill'].unique():
    df2b_dict[skill] = skill_extract(df2b, skill).drop_duplicates(subset='ID', keep='first') # Keep the first since all duplicates after the first coincidentally only have missing/Nan values

print('A complete list of skills:', df2b_dict.keys())
    
#Uncomment the line below to show an example of a df_slice
#df2b_dict['Chart Interpretation'] # This will have no duplicate Candidate Test Ids

## Joining all the dataframes in the dictionary 'df1b_dict' on Candidate Test Id
df2b = df2b[['ID']].drop_duplicates()
for key in df2b_dict:
    df2b = df2b.merge(df2b_dict[key], how='outer', on='ID', sort=True)

## Drop duplicate rows
df2b = df2b.drop_duplicates()

print('df2b has', len(df2b), 'rows and', len(df2b['ID'].unique()), 'unique IDs')

df2b.to_csv('Senior_Commercial_Analyst_(Amsterdam)_Skills_New.csv') #Save dataframe as csv

#df2b

TypeError: cannot astype a datetimelike from [datetime64[ns]] to [float64]

In [8]:
######################################
####### COMBINE DF2B AND DF2A ########
######################################

Senior_Analyst = df2a.merge(df2b, how='outer', on='ID')
Senior_Analyst.to_csv('Senior_Commercial_Analyst_(Amsterdam)_All.csv')

# View Duplicate IDs
Senior_Analyst[Senior_Analyst.duplicated(subset='ID', keep=False)]

,Have CV,ID,Location,Date Invited,Date Started,Date Completed,Parts Completed,Overall Score,Business Acumen score,Business Acumen Self Rating,Chart Interpretation score,Chart Interpretation Self Rating,Data Literacy score,Data Literacy Self Rating,Reports & Visualisations score,Reports & Visualisations Self Rating,SQL score,SQL Self Rating
0,NaN,26928,France,2021-08-29 10:40:04,2021-09-03 18:29:23,2021-09-03 19:32:14,3.0,4.3,NaN,8.0,6.19,7.0,6.818,5.0,5.0,6.0,NaN,6.0
1,NaN,26928,United Arab Emirates,2021-08-29 11:04:14,2021-09-03 02:14:01,2021-09-03 03:04:10,3.0,1.7,NaN,8.0,6.19,7.0,6.818,5.0,5.0,6.0,NaN,6.0


# Updated Senior Commercial Analyst Merging

In [60]:
import pandas as pd
dfn1 = pd.read_csv('Senior-Commercial-Analyst-Amsterdam.csv')
dfn3 = pd.read_csv('Senior-Commercial-Analyst-Amsterdam_skills.csv')

In [61]:
######################################
################ DFN1 ################
######################################

dfn1['Date Invited'] = pd.to_datetime(dfn1['Date Invited']).dt.tz_localize(None)
dfn1['Date Started'] = pd.to_datetime(dfn1['Date Started']).dt.tz_localize(None)
dfn1['Date Completed'] = pd.to_datetime(dfn1['Date Completed']).dt.tz_localize(None)
dfn1['Overall Score'] = dfn1['Overall Score'].str.rstrip('%').astype('float') / 100.0
dfn1['Overall Score'] = dfn1['Overall Score']*10

dfn1.to_csv('Senior-Commercial-Analyst-(Amsterdam)-Candidates.csv') #Save dataframe as csv

In [62]:
######################################
################ DFN3 ################
######################################

##Function that returns a dataframe filtered by the skill (i.e. Data Literacy, Statistics, SQL, ... )
##Same as the one in df1b
def skill_extract(df, skill):
    df_slice = df[df['Skill'] == skill]
    df_slice = df_slice.rename(columns={'Score':skill+' score', 'Self Rating':skill+' Self Rating'}) #Rename columns for interpretability
    df_slice = df_slice.drop(columns='Skill')
    df_slice = df_slice.replace(to_replace='(\d+)\/10', value='\\1', regex=True) #Changes x/10 to numeric (just x)
    df_slice = df_slice.replace(to_replace='([0-9\.]+)\%', value='\\1', regex=True) #Removes percentage sign
    df_slice = df_slice.astype({skill+' score': 'float', skill+' Self Rating':'float'}) #Convert columns to float
    df_slice[skill+' score'] = df_slice[skill+' score']/10 #Convert percentage to score out of 10
    return df_slice

##Create dfs, a dictionary of dataframes with keys being the skill and values being the dataframe filtered by skill
dfn3_dict = {}
for skill in dfn3['Skill'].unique():
    df2b_dict[skill] = skill_extract(dfn3, skill).drop_duplicates(subset='ID', keep='first') # Keep the first since all duplicates after the first coincidentally only have missing/Nan values

print('A complete list of skills:', df2b_dict.keys())
    
#Uncomment the line below to show an example of a df_slice
#df2b_dict['Chart Interpretation'] # This will have no duplicate Candidate Test Ids

## Joining all the dataframes in the dictionary 'df1b_dict' on Candidate Test Id
dfn3 = dfn3[['ID']].drop_duplicates()
for key in df2b_dict:
    dfn3 = dfn3.merge(df2b_dict[key], how='outer', on='ID', sort=True)

## Drop duplicate rows
dfn3 = dfn3.drop_duplicates()

print('df2b has', len(dfn3), 'rows and', len(dfn3['ID'].unique()), 'unique IDs')

dfn3.to_csv('Senior-Commercial-Analyst-(Amsterdam)-Skills.csv') #Save dataframe as csv

A complete list of skills: dict_keys(['Chart Interpretation', 'Business Acumen', 'Reports & Visualisations', 'SQL', 'Data Literacy'])
df2b has 1126 rows and 1126 unique IDs


In [63]:
######################################
####### COMBINE DFN1 AND DFN3 ########
######################################

Senior_Analyst = dfn1.merge(dfn3, how='outer', on='ID')
Senior_Analyst.to_csv('Senior-Commercial-Analyst-(Amsterdam)-All.csv')

# View Duplicate IDs
Senior_Analyst[Senior_Analyst.duplicated(subset='ID', keep=False)]

,Have CV,ID,Location,Date Invited,Date Started,Date Completed,Parts Completed,Overall Score,Chart Interpretation score,Chart Interpretation Self Rating,Business Acumen score,Business Acumen Self Rating,Reports & Visualisations score,Reports & Visualisations Self Rating,SQL score,SQL Self Rating,Data Literacy score,Data Literacy Self Rating
152,1.0,28721,Pakistan,2021-09-13 13:41:28,2021-09-18 14:45:03,2021-09-18 15:50:43,3,3.8,4.762,9.0,0.0,7.0,5.385,9.0,0.0,8.0,7.273,8.0
153,1.0,28721,Pakistan,2021-09-13 13:41:28,2021-09-18 14:45:03,2021-09-18 15:50:43,3,3.8,4.762,9.0,0.0,7.0,5.385,9.0,0.0,8.0,7.273,8.0


In [64]:
Senior_Analyst

,Have CV,ID,Location,Date Invited,Date Started,Date Completed,Parts Completed,Overall Score,Chart Interpretation score,Chart Interpretation Self Rating,Business Acumen score,Business Acumen Self Rating,Reports & Visualisations score,Reports & Visualisations Self Rating,SQL score,SQL Self Rating,Data Literacy score,Data Literacy Self Rating
0,1.0,27318,Brazil,2021-08-30 20:24:12,2021-09-02 09:50:41,2021-09-02 10:47:17,3,8.7,8.571,8.0,4.286,6.0,9.0,7.0,8.750,10.0,10.000,9.0
1,1.0,27613,India,2021-09-02 20:35:14,2021-09-06 21:12:52,2021-09-06 22:13:37,3,8.4,9.048,10.0,8.571,9.0,8.0,9.0,7.500,8.0,8.636,10.0
2,1.0,28057,South Africa,2021-09-07 14:31:40,2021-09-08 04:01:36,2021-09-08 04:53:03,3,8.0,7.619,8.0,4.286,8.0,9.0,8.0,9.375,8.0,8.182,8.0
3,1.0,28257,Iran,2021-09-09 15:21:08,2021-09-12 17:45:58,2021-09-12 19:11:26,3,8.0,10.000,9.0,8.571,8.0,6.0,9.0,5.000,7.0,9.091,10.0
4,1.0,27223,Malta,2021-08-29 11:05:22,2021-08-29 23:46:52,2021-08-30 00:34:05,3,7.9,8.095,9.0,8.571,9.0,9.0,9.0,3.750,9.0,10.000,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1122,NaN,28964,NaN,2021-09-15 11:32:23,NaT,NaT,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1123,NaN,28965,NaN,2021-09-15 11:32:24,NaT,NaT,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1124,NaN,28967,NaN,2021-09-15 11:32:25,NaT,NaT,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1125,NaN,28970,NaN,2021-09-15 11:32:27,NaT,NaT,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [92]:
#print(len(df1a['Date Invited'].unique()))
#df1a.groupby('Date Invited').nunique().sort_values(by='Candidate Test Id').tail(50)
#df1a.groupby('Date Invited').nunique()[df1a.groupby('Date Invited').nunique()['Candidate Test Id'] == 1][1:100]